# A program to using Dynamic Programming

Using Dynamic Programming to generate Buckets as part of the JPMorgan Job Simulation Task 4 assignment. 

## Methodology

- Manually assessing the FICO scores, seeing the least data eyeballing what to split it up by
- Grouped by chunks of 50
- Calculated default rate for each chuck
- Looked at where there were dramatic drops for each chunk
- Drew a boundary line at those drop points
- Calculated the MSE to see if the buckets were good
- Tried different boundary lines to see which was the best

Then used OptBinning to do it automatically and returned roughly the same thing. 

The MSE is significant because the closer it is to 0 the more likely it is that everyone in that bucket behaves the same way. 


# OptBinning Library

In [9]:
!pip install optbinning -q

In [11]:
from optbinning import OptimalBinning
import pandas as pd
import numpy as np


In [12]:
pd.set_option('display.max_columns', None) # Set option to display all columns
pd.set_option('display.float_format', '{:.2f}'.format) # Set float format to 2 decimal places

df = pd.read_csv('Task 3 and 4_Loan_Data.csv')
df.info()
display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               10000 non-null  int64  
 1   credit_lines_outstanding  10000 non-null  int64  
 2   loan_amt_outstanding      10000 non-null  float64
 3   total_debt_outstanding    10000 non-null  float64
 4   income                    10000 non-null  float64
 5   years_employed            10000 non-null  int64  
 6   fico_score                10000 non-null  int64  
 7   default                   10000 non-null  int64  
dtypes: float64(3), int64(5)
memory usage: 625.1 KB


,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.55,3915.47,78039.39,5,605,0
1,7442532,5,1958.93,8228.75,26648.44,2,572,1
2,2256073,0,3363.01,2027.83,65866.71,4,602,0
3,4885975,0,4766.65,2501.73,74356.88,5,612,0
4,4700614,1,1345.83,1768.83,23448.33,6,631,0


In [13]:
# explore the fico score variable given we are trying to create buckets for it
print(df['fico_score'].describe())
df['fico_range'] = (df['fico_score'] // 50) * 50
default_by_range = df.groupby('fico_range')['default'].mean() # calculate percentage of defaults by taking the mean of the default column
print(default_by_range)

count   10000.00
mean      637.56
std        60.66
min       408.00
25%       597.00
50%       638.00
75%       679.00
max       850.00
Name: fico_score, dtype: float64
fico_range
400   0.70
450   0.72
500   0.49
550   0.30
600   0.18
650   0.10
700   0.05
750   0.03
800   0.03
850   0.00
Name: default, dtype: float64


Based on the data,
 Deny risk is 400-450 
 High Risk is 451-550
 Medium Risk is 551 - 700
 Low Risk is 701 -850

In [ ]:
def assign_bucket(fico):  
    if fico <= 450:
        return 'Deny'
    elif fico <= 550:
        return 'High Risk'
    elif fico <= 700:
        return 'Medium Risk'
    else:
        return 'Low Risk'

df['bucket'] = df['fico_score'].apply(assign_bucket)
display(df[['fico_score', 'bucket']].head(10))


,fico_score,bucket
0,605,Medium Risk
1,572,Medium Risk
2,602,Medium Risk
3,612,Medium Risk
4,631,Medium Risk
5,697,Medium Risk
6,722,Low Risk
7,545,High Risk
8,676,Medium Risk
9,447,Deny


In [32]:
bucket_rates = df.groupby('bucket')['default'].mean()
print(bucket_rates)

bucket
Deny          0.58
High Risk     0.54
Low Risk      0.05
Medium Risk   0.18
Name: default, dtype: float64


In [33]:
df['predicted'] = df['bucket'].map(bucket_rates)
df['squared_error'] = (df['default'] - df['predicted']) ** 2
mse = df['squared_error'].mean()
print(f"MSE: {mse:.4f}")

MSE: 0.1382


In [ ]:
def assign_bucket_v2(fico):  # seeing if changing the buckets improves MSE
    if fico <= 550:
        return 'High Risk'
    elif fico <= 600:
        return 'Medium-High Risk'
    elif fico <= 700:
        return 'Medium-Low Risk'
    else:
        return 'Low Risk'
df['bucket'] = df['fico_score'].apply(assign_bucket_v2)
display(df[['fico_score', 'bucket']].head(10))

,fico_score,bucket
0,605,Medium-Low Risk
1,572,Medium-High Risk
2,602,Medium-Low Risk
3,612,Medium-Low Risk
4,631,Medium-Low Risk
5,697,Medium-Low Risk
6,722,Low Risk
7,545,High Risk
8,676,Medium-Low Risk
9,447,High Risk


In [27]:
bucket_rates = df.groupby('bucket')['default'].mean()
print(bucket_rates)

bucket
High Risk          0.54
Low Risk           0.05
Medium-High Risk   0.29
Medium-Low Risk    0.14
Name: default, dtype: float64


In [30]:
df['predicted'] = df['bucket'].map(bucket_rates)
df['squared_error'] = (df['default'] - df['predicted']) ** 2
mse = df['squared_error'].mean()
print(f"MSE: {mse:.4f}")

MSE: 0.1349


Now doing the same thing with a optbinning finding the best possible mse  because manually takes too long

In [35]:
from optbinning import OptimalBinning

# X = FICO scores, y = default status
optb = OptimalBinning(name='fico_score', dtype='numerical', max_n_bins = 4)
optb.fit(df['fico_score'].values, df['default'].values)

# See the boundaries
print("Optimal binning thresholds:", optb.splits)
print(optb.binning_table.build())

Optimal binning thresholds: [550.5 610.5 696.5]
                     Bin  Count  Count (%)  Non-event  Event  Event rate  \
0         (-inf, 550.50)    758       0.08        347    411        0.54   
1       [550.50, 610.50)   2443       0.24       1752    691        0.28   
2       [610.50, 696.50)   5142       0.51       4470    672        0.13   
3          [696.50, inf)   1657       0.17       1580     77        0.05   
4                Special      0       0.00          0      0        0.00   
5                Missing      0       0.00          0      0        0.00   
Totals                    10000       1.00       8149   1851        0.19   

         WoE   IV   JS  
0      -1.65 0.30 0.03  
1      -0.55 0.09 0.01  
2       0.41 0.08 0.01  
3       1.54 0.23 0.03  
4       0.00 0.00 0.00  
5       0.00 0.00 0.00  
Totals       0.69 0.08  
